# 向量检索实现
> By SherryAGI | 语义搜索与混合检索

## 向量检索 vs 关键词检索

| 特性 | 关键词检索 | 向量检索 |
|------|------------|----------|
| 原理 | 精确匹配 | 语义相似 |
| 同义词 | ❌ 无法识别 | ✅ 自动理解 |
| 速度 | ⚡ 极快 | 🔄 较快 |
| 适用 | 精确查找 | 模糊搜索 |

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

print('🚀 向量检索模块 by SherryAGI')

## 1. Embedding 模型选择

In [ ]:
# 推荐的 Embedding 模型
embedding_models = {
    "多语言": [
        "BAAI/bge-m3",
        "intfloat/multilingual-e5-large"
    ],
    "中文": [
        "BAAI/bge-large-zh-v1.5",
        "shibing624/text2vec-base-chinese"
    ],
    "英文": [
        "BAAI/bge-large-en-v1.5",
        "sentence-transformers/all-MiniLM-L6-v2"
    ],
    "API": [
        "OpenAI text-embedding-3-small",
        "Cohere embed-multilingual-v3.0"
    ]
}

print('📊 推荐 Embedding 模型:')
for category, models in embedding_models.items():
    print(f'\n{category}:')
    for m in models:
        print(f'  - {m}')

## 2. 向量数据库实现

In [ ]:
class SimpleVectorDB:
    """简单向量数据库 by SherryAGI"""
    
    def __init__(self, embedding_dim=384):
        self.documents = []
        self.embeddings = []
        self.dim = embedding_dim
    
    def _mock_embed(self, text):
        """模拟向量化"""
        np.random.seed(hash(text) % 2**32)
        return np.random.rand(self.dim)
    
    def add(self, text, metadata=None):
        """添加文档"""
        embedding = self._mock_embed(text)
        self.documents.append({"text": text, "metadata": metadata or {}})
        self.embeddings.append(embedding)
        return len(self.documents) - 1
    
    def search(self, query, top_k=5):
        """向量搜索"""
        query_embedding = self._mock_embed(query)
        
        similarities = cosine_similarity(
            [query_embedding], 
            self.embeddings
        )[0]
        
        # 排序
        indices = np.argsort(similarities)[::-1][:top_k]
        
        results = []
        for idx in indices:
            results.append({
                "score": float(similarities[idx]),
                "text": self.documents[idx]["text"],
                "metadata": self.documents[idx]["metadata"]
            })
        
        return results

print('✅ SimpleVectorDB 已定义')

## 3. 测试向量检索

In [ ]:
# 创建数据库
db = SimpleVectorDB()

# 添加企业知识
docs = [
    ("报销流程：提交发票，填写报销单，主管审批", {"category": "财务"}),
    ("请假申请：登录OA系统，选择请假类型，提交审批", {"category": "HR"}),
    ("会议室预订：通过企业微信预订，最多提前7天", {"category": "行政"}),
    ("VPN连接：下载客户端，输入账号密码，选择服务器", {"category": "IT"}),
    ("密码重置：联系IT热线8888或自助重置", {"category": "IT"}),
]

for text, meta in docs:
    db.add(text, meta)

print(f'✅ 已添加 {len(docs)} 条记录')

In [ ]:
# 测试搜索
queries = [
    "怎么报销？",
    "忘记密码了",
    "想请假"
]

for query in queries:
    print(f'\n🔍 查询: {query}')
    results = db.search(query, top_k=2)
    for i, r in enumerate(results):
        print(f'  {i+1}. [{r["metadata"]["category"]}] {r["text"][:40]}... (相似度: {r["score"]:.3f})')

## 4. 混合检索

In [ ]:
class HybridSearch:
    """混合检索：向量 + 关键词 by SherryAGI"""
    
    def __init__(self):
        self.vector_db = SimpleVectorDB()
        self.keyword_index = {}  # 简单倒排索引
    
    def add(self, text, metadata=None):
        idx = self.vector_db.add(text, metadata)
        # 建立关键词索引
        for word in text.split():
            if word not in self.keyword_index:
                self.keyword_index[word] = []
            self.keyword_index[word].append(idx)
        return idx
    
    def search(self, query, top_k=5, alpha=0.7):
        """混合搜索 (alpha: 向量权重)"""
        # 向量搜索
        vector_results = self.vector_db.search(query, top_k * 2)
        
        # 关键词搜索
        keyword_scores = {}
        for word in query.split():
            if word in self.keyword_index:
                for idx in self.keyword_index[word]:
                    keyword_scores[idx] = keyword_scores.get(idx, 0) + 1
        
        # 融合分数
        final_scores = {}
        for r in vector_results:
            idx = self.vector_db.documents.index(
                {"text": r["text"], "metadata": r["metadata"]}
            )
            vector_score = r["score"]
            keyword_score = keyword_scores.get(idx, 0) / max(len(query.split()), 1)
            final_scores[idx] = alpha * vector_score + (1 - alpha) * keyword_score
        
        # 排序返回
        sorted_indices = sorted(final_scores.keys(), key=lambda x: -final_scores[x])
        
        results = []
        for idx in sorted_indices[:top_k]:
            doc = self.vector_db.documents[idx]
            results.append({
                "score": final_scores[idx],
                "text": doc["text"],
                "metadata": doc["metadata"]
            })
        
        return results

print('✅ HybridSearch 已定义')

---
**By SherryAGI** | [DigitalTransformationAI](https://github.com/AIB612/DigitalTransformationAI)